# Wrangling Data from OpenStreetMap for the Greater Salt Lake City Area

I have used Mapzen (https://mapzen.com/) to extract a dataset from OpenStreetMap. The area I have chosen is Greater Salt Lake City. The total uncompressed size of this file is 318 MB.   

## 1. Processing the Dataset

In [28]:
import csv
import pprint
import xml.etree.ElementTree as ET  

OSM_FILE = "SaltLakeMap.osm"  
SAMPLE_FILE = "sample.osm"

k = 25 # Take every 25th top level element

def get_element(OSM_FILE, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(OSM_FILE, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

The above code takes a sample of the full OSM dataset for the Greater Salt Lake City area so we can explore a smaller, more managable amount of data. This will save time by allowing us to test our scripts before attempting to wrangle the entire 381 MB dataset. The sample.osm file we created with the above script is 6.44 MB. 

### Auditing the Data

In [29]:
tags = {}

def count_tags(SAMPLE_FILE):
        
        for event, elem in ET.iterparse(SAMPLE_FILE):
            if elem.tag in tags.keys():
                tags[elem.tag]+= 1
            else:
                tags[elem.tag] = 1
                
count_tags(SAMPLE_FILE)        
print tags

{'node': 61085, 'nd': 70493, 'member': 219, 'tag': 30395, 'relation': 42, 'way': 7302, 'osm': 1}


This code tells us which data primitives there are in the SAMPLE_FIlE and gives us a sense of how many of each data primitive we can expect in the full file. An explanation of these data primitives can be found here: https://wiki.openstreetmap.org/wiki/OSM_XML

In [30]:
import re
import xml.etree.cElementTree as ET
import pprint

lower = re.compile(r'^([a-z]|_)*$') # for tags that contain only lowercase letters and are valid
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$') # for otherwise valid tags with a colon in their names
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]') # for tags with problematic characters 

def key_type(element, keys):
    if element.tag == "tag":
        k = element.get("k")
        if re.search(lower,k):
            keys['lower'] +=1
        elif re.search(lower_colon,k):
            keys['lower_colon'] +=1
        elif re.search(problemchars,k):
            keys['problemchars'] +=1
        else:
            keys['other'] +=1
    
    return keys

def process_map(SAMPLE_FILE):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for event, element in ET.iterparse(SAMPLE_FILE, events=("start",)):
        keys = key_type(element, keys)
        
    print keys
        
process_map(SAMPLE_FILE)

{'problemchars': 0, 'lower': 15834, 'other': 3317, 'lower_colon': 11244}


The above code loops through all of the tags named "tag" rather than only those tags named "tag" within the "ways" tag as the code below does. 

In [31]:
import re
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict

lower = re.compile(r'^([a-z]|_)*$') 
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$') 
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')  

def key_type(element, keys):
    if element.tag == "way":
        for tag in element.iter("tag"):
            k = tag.get("k")
            if lower.search(k):
                keys['lower'] +=1
            elif lower_colon.search(k):
                keys['lower_colon'] +=1
            elif problemchars.search(k):
                keys['problemchars'] +=1
            else:
                keys['other'] +=1
    
    return keys
              
def process_map(SAMPLE_FILE):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for event, element in ET.iterparse(SAMPLE_FILE, events=("start",)):
        keys = key_type(element, keys)
        
    print keys
        
process_map(SAMPLE_FILE)

def test():
    keys = process_map(SAMPLE_FILE)


{'problemchars': 0, 'lower': 12677, 'other': 2647, 'lower_colon': 10469}


This piece of code takes SAMPLE_FILE and uses iterparse to find all of the top-level "start" tags titled "way." Then within this "way" tags, the script finds the tags labeled "tag" and gets the k values within these tags to see if the values within those tags are address values and don't contain problem characters. The script then checks whether the k values include only lowercase letters, lowercase letters with colons, or a predefined set of problem characters. The script then adds the number of elements that meet each on of these constraints in a dictionary titled "keys" and outputs that dictionary.        

In [32]:
def get_user(element):
    if element.tag == "way":
       user = element.get("uid")

       return user
    
def process_map(SAMPLE_FILE):
    users = set()
    for event, element in ET.iterparse(SAMPLE_FILE):
        if get_user(element):
           users.add(get_user(element))       
       
    print len(users)

process_map(SAMPLE_FILE)

405


The code above counts the number of unique users who have contributed to the "way" tags for OpenStreetMap for Greater Salt Lake City. 

### Cleaning the Data and Converting it to the CSV Format

In [33]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint


street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE) #pulls out last word when tag attribute is "v"


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", "West", "East", "North", "South", "Temple", "Way", "Gateway", "Broadway"]

mapping = { "Rd.": "Road",
            "1300": "",
            "N": "North",
            "319": "",
            "A": ""
            }
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(SAMPLE_FILE):
    open(SAMPLE_FILE, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(SAMPLE_FILE, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    
    return street_types

audit(SAMPLE_FILE)

defaultdict(set,
            {'1300': {'1300'},
             '319': {'S Sleepy Ridge Drive Suite 319'},
             'A': {'S State St #A'},
             'N': {'W 800 N'},
             'Rd.': {'Portobello Rd.', 'West Portobello Rd.'}})

When I first ran this auditing script, I found a lot of addresses that ended with words that were not in the "expected" list but which are perfectly acceptable for Salt Lake City. These include addresses ending with "North,"South," "East," and "West." I also found some ending with "Circle" (and others), which are perfectly acceptable. Thus, I have added these words to the expected list above so that they will be ignored. 

In my sample file, I am only left with one incorrect street name—one that ends in "Rd." instead of "Road."

In [34]:
def update_name(name, mapping):

    m = street_type_re.search(name)
    if m.group() not in expected:
        if m.group() in mapping.keys():
            name = re.sub(m.group(), mapping[m.group()], name)
    return name

def test():
    st_types = audit(SAMPLE_FILE)
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.iteritems(): 
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
           
test()

{'1300': set(['1300']),
 '319': set(['S Sleepy Ridge Drive Suite 319']),
 'A': set(['S State St #A']),
 'N': set(['W 800 N']),
 'Rd.': set(['Portobello Rd.', 'West Portobello Rd.'])}
S State St #A => S State St #
S Sleepy Ridge Drive Suite 319 => S Sleepy Ridge Drive Suite 
Portobello Rd. => Portobello Road
West Portobello Rd. => West Portobello Road
1300 => 
W 800 N => W 800 North


Here, we can see how the street names will be cleaned when the are inserted into the CSV files (which comes below).

In [35]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema
from audit import *  

OSM_PATH = "sample.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema


NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  
    

    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]
                
        for tag in element.iter("tag"):
            node_tags_attribs = {}
            if LOWER_COLON.search(tag.attrib['k']):
                node_tags_attribs['id'] = element.attrib['id']
                node_tags_attribs['key'] = tag.attrib['k'].split(':',1)[1]
                node_tags_attribs['value'] = tag.attrib['v']
                node_tags_attribs['type'] = tag.attrib['k'].split(':',1)[0]
                if tag.attrib['k'] == 'addr:street':
                    node_tags_attribs['value'] = update_name(tag.attrib['v'], mapping)
                else:
                    node_tags_attribs['value'] = tag.attrib['v'] 
                tags.append(node_tags_attribs) 
              
            elif PROBLEMCHARS.search(tag.attrib['k']):
                continue 
                
            else:
                node_tags_attribs['id'] = element.attrib['id']
                node_tags_attribs['key'] = tag.attrib['k']
                node_tags_attribs['value'] = tag.attrib['v']
                node_tags_attribs['type'] = 'regular'
                tags.append(node_tags_attribs)
                
            
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]

        position = 0        
        for child in element:
            way_node = {}
            way_tag = {}

            if  child.tag == 'tag':
                if LOWER_COLON.match(child.attrib['k']):
                    way_tag['id'] = element.attrib['id']
                    way_tag['type'] = child.attrib['k'].split(':',1)[0]
                    way_tag['key'] = child.attrib['k'].split(':',1)[1]
                    way_tag['value'] = child.attrib['v']
                    if child.attrib['k'] == 'addr:street':
                        way_tag['value'] = update_name(child.attrib['v'], mapping)
                    else:
                        way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                   
                elif PROBLEMCHARS.match(child.attrib['k']):
                       continue

                else:
                        way_tag['type'] = 'regular'
                        way_tag['id'] = element.attrib['id']
                        way_tag['key'] = child.attrib['k']
                        way_tag['value'] = child.attrib['v']
                        tags.append(way_tag)

            elif  child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = position
                position += 1
                way_nodes.append(way_node) 
    
    return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


def get_element(SAMPLE_FILE, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(SAMPLE_FILE, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)



def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    
    process_map(OSM_PATH, validate=True)

This code takes all the data from the XML, cleans it, and puts it into CSV files that can then be added to an SQL database (which is done below).

## 2. Exploring the Database

In [36]:
import sqlite3
import csv
from pprint import pprint

sqlite_file = 'SQL_Salt_Lake.db'

conn = sqlite3.connect(sqlite_file)

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS nodes_tags''')
conn.commit()

cur.execute('''CREATE TABLE nodes_tags(id INTEGER, key TEXT, value TEXT, type TEXT, FOREIGN KEY(id) REFERENCES nodes(id))''');
conn.commit()

with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr]
    

cur.executemany("INSERT INTO nodes_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)

conn.commit()

# cur.execute('SELECT * FROM nodes_tags')
# all_rows = cur.fetchall()
# print('1):')
# pprint(all_rows)

With the above code, I have created a table called "nodes_tags" in the SQL database titled "SQL_Salt_Lake" and inserted the data from nodes_tags.csv into it. 

In [37]:
import sqlite3
import csv
from pprint import pprint

sqlite_file = 'SQL_Salt_Lake.db'

conn = sqlite3.connect(sqlite_file)

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS nodes''')
conn.commit()

cur.execute('''CREATE TABLE nodes(id INTEGER PRIMARY KEY NOT NULL, lat REAL, lon REAL, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT)''');
conn.commit()

with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'].decode("utf-8"), i['lat'].decode("utf-8"), i['lon'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8")) for i in dr]
    

cur.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)

conn.commit()

# cur.execute('SELECT * FROM nodes')
# all_rows = cur.fetchall()
# print('1):')
# pprint(all_rows)

With the above code, I have created a table called "nodes" in the SQL database titled "SQL_Salt_Lake" and inserted the data from nodes.csv into it. 

In [38]:
import sqlite3
import csv
from pprint import pprint

sqlite_file = 'SQL_Salt_Lake.db'

conn = sqlite3.connect(sqlite_file)

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS ways''')
conn.commit()

cur.execute('''CREATE TABLE ways(id INTEGER PRIMARY KEY NOT NULL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT)''');
conn.commit()

with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8")) for i in dr]
    

cur.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)

conn.commit()

# cur.execute('SELECT * FROM ways')
# all_rows = cur.fetchall()
# print('1):')
# pprint(all_rows)

With the above code, I have created a table called "ways" in the SQL database titled "SQL_Salt_Lake" and inserted the data from ways.csv into it. 

In [39]:
import sqlite3
import csv
from pprint import pprint

sqlite_file = 'SQL_Salt_Lake.db'

conn = sqlite3.connect(sqlite_file)

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS ways_tags''')
conn.commit()

cur.execute('''CREATE TABLE ways_tags(id INTEGER NOT NULL, key TEXT NOT NULL, value TEXT NOT NULL, type TEXT, FOREIGN KEY (id) REFERENCES ways(id))''');
conn.commit()

with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr]
    

cur.executemany("INSERT INTO ways_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db)

conn.commit()

# cur.execute('SELECT * FROM ways_tags')
# all_rows = cur.fetchall()
# print('1):')
# pprint(all_rows)

With the above code, I have created a table called "ways_tags" in the SQL database titled "SQL_Salt_Lake" and inserted the data from ways_tags.csv into it. 

In [40]:
import sqlite3
import csv
from pprint import pprint

sqlite_file = 'SQL_Salt_Lake.db'

conn = sqlite3.connect(sqlite_file)

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS ways_nodes''')
conn.commit()

cur.execute('''CREATE TABLE ways_nodes(id INTEGER NOT NULL, node_id INTEGER NOT NULL, position INTEGER NOT NULL, FOREIGN KEY (id) REFERENCES ways(id), FOREIGN KEY (node_id) REFERENCES nodes (id))''');
conn.commit()

with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'].decode("utf-8"), i['node_id'].decode("utf-8"), i['position'].decode("utf-8")) for i in dr]
    

cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db)

conn.commit()

# cur.execute('SELECT * FROM ways_nodes');
# all_rows = cur.fetchall()
# print('1):')
# pprint(all_rows)

With the above code, I have created a table called "ways_nodes" in the SQL database titled "SQL_Salt_Lake" and inserted the data from ways_nodes.csv into it. 

In [41]:
cur.execute('SELECT COUNT(*) FROM nodes');
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

1):
[(61085,)]


Here, I am counting the number of tags in the "nodes" table. 

In [42]:
cur.execute('SELECT COUNT(*) FROM nodes_tags');
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

1):
[(3708,)]


Here, I am counting the number of tags in the "nodes_tags" table. 

In [43]:
cur.execute('SELECT COUNT(*) FROM ways');
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

1):
[(7302,)]


Here, I am counting the number of tags in the "ways" table. 

In [44]:
cur.execute('SELECT COUNT(*) FROM ways_tags');
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

1):
[(26480,)]


Here, I am counting the number of tags in the "ways_tags" table. 

In [45]:
cur.execute('SELECT COUNT(*) FROM ways_nodes');
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

1):
[(70493,)]


Here, I am counting the number of tags in the "ways_nodes" table. 

In [46]:
cur.execute('SELECT COUNT(DISTINCT(uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways)');
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

1):
[(794,)]


Here, I am counting the number of unique users who contributed to the data in the OSM area I selected.

In [47]:
cur.execute('SELECT user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) GROUP BY user ORDER BY num DESC LIMIT 20');
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

1):
[(u'chadbunn', 9895),
 (u'osmjwh', 6360),
 (u'woodpeck_fixbot', 4288),
 (u'butlerm', 3495),
 (u'mash84121', 2866),
 (u'mvexel', 2747),
 (u'MelanieOriet', 2612),
 (u'Level', 2006),
 (u'carlotta4th', 1963),
 (u'OremSteve', 1735),
 (u'wrk3', 1614),
 (u'Ted Percival', 1387),
 (u'MasterOfKittens', 1305),
 (u'Val', 1259),
 (u'jackwiplock', 927),
 (u'TheDutchMan13', 903),
 (u'GaryOSM', 883),
 (u'nemmer', 799),
 (u'balcoath', 731),
 (u'bburgon42', 646)]


Here, I am counting the number of data entries each user contributed to the data in the OSM area I selected.


In [48]:
cur.execute('SELECT value, COUNT(*) as num FROM nodes_tags WHERE key="amenity" GROUP BY value ORDER BY num DESC LIMIT 10');
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

1):
[(u'restaurant', 41),
 (u'place_of_worship', 32),
 (u'fast_food', 19),
 (u'fuel', 14),
 (u'parking', 14),
 (u'bank', 8),
 (u'bench', 8),
 (u'toilets', 8),
 (u'school', 6),
 (u'cafe', 5)]


Here, I am looking the number of each type of amenity found in the OSM area I selected. 

In [49]:
cur.execute('SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags WHERE nodes_tags.key="religion" GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 5');
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

1):
[(u'christian', 31)]


Here, I am looking at the number of places of worship by each religion type in the OSM area I selected.


In [50]:
cur.execute('SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags WHERE nodes_tags.key="denomination" GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 5');
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

1):
[(u'mormon', 22),
 (u'latter_day_saints', 2),
 (u'catholic', 1),
 (u'jehovahs_witness', 1)]


Here, I am looking at the denominations represented in the places of worship in the OSM area I selected.


In [51]:
cur.execute('SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags WHERE nodes_tags.key="name" GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 5');
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

1):
[(u'The Church of Jesus Christ of Latter-day Saints', 22),
 (u'7-11', 3),
 (u'7-Eleven', 3),
 (u"Arby's", 3),
 (u'Burger King', 2)]


Here, I was interested to see the names of the facilities of the places in my dataset. 

In [52]:
cur.execute('SELECT tags.value, COUNT(*) as xyz FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) tags WHERE tags.key="postcode" GROUP BY tags.value ORDER BY xyz DESC LIMIT 10');
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)

1):
[(u'84105', 219),
 (u'84108', 127),
 (u'84106', 123),
 (u'84102', 56),
 (u'84116', 14),
 (u'84005', 11),
 (u'84096', 7),
 (u'84109', 7),
 (u'84084', 6),
 (u'84103', 5)]


And here, I wanted to see where most of the places in my OSM file were located. It looks like they are mainly concentrated around four zip codes.

## 3. Conclusions

I chose to look at every 25th data primitive in the OSM XML for Greater Salt Lake City because the full dataset crashed my computer when I tried to analyze it. This full file was .33 GB. The sample file was .013 GB. 

Here are a few interesting things I found about the data:

- 794 unique users contributed data to the sample of the OSM XML I looked at
- A single user, "chadbunn" contributed 20% of entries of the top 20 contibutors (9895/48420). 
- The most common amenity is a "restaurant"(41), which is followed by "places of worship" (32).
- The only religion found at these places of worship is Christianity.
- The denominations of these Christians are Mormon (24), Catholic (1), and Jehovas Witness (1). This seems reasonable considering Salt Lake City is the geographic center of the Mormon faith.
- In the area under investigation, most of the names of the places are "The Church of Jesus Christ of Latter-day Saints," and the rest are convienience stores and fast-food chains.
- There are no zip codes in the sample that seem to be erroneous; they all appear to be in areas within the OSM XML area that was extracted. 


An area for further investigation would be to run an analysis on the entire data set and see how the religious makeup of the geographic area captured by the data in the OSM XLM aligns up with official statistics for religious affiliation for Salt Lake City and for Utah as a whole.  
